<a href="https://colab.research.google.com/github/Alan-Cheong/IEEE_QW_2020/blob/master/Bayesian_Optimisation_Grok3_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def squared_exponential_kernel(x, x_prime, length_scale=0.3, sigma_f=1.0):
    """
    Squared exponential kernel: k(x, x') = sigma_f^2 * exp(-||x - x'||^2 / (2 * length_scale^2))
    """
    sq_dist = np.sum((x - x_prime) ** 2)
    return sigma_f ** 2 * np.exp(-sq_dist / (2 * length_scale ** 2))

def compute_k_x_X(x, X, length_scale=0.3, sigma_f=1.0):
    """
    Compute k(x, X): covariance vector between new point x and observed points X.
    """
    return np.array([squared_exponential_kernel(x, x_i, length_scale, sigma_f) for x_i in X])

def compute_K(X, length_scale=0.3, sigma_f=1.0, noise_variance=0.0):
    """
    Compute K: covariance matrix for observed points X.
    """
    n = len(X)
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i, j] = squared_exponential_kernel(X[i], X[j], length_scale, sigma_f)
    if noise_variance > 0:
        K += noise_variance * np.eye(n)
    return K

def gp_predict(x, X, y, length_scale=0.3, sigma_f=1.0, noise_variance=0.0):
    """
    Compute GP predictive mean and standard deviation at point x.
    Inputs:
        x: New point (scalar or 1D array).
        X: Observed points (array of scalars or 1D arrays).
        y: Observed values (array).
        length_scale: Kernel length scale.
        sigma_f: Signal variance.
        noise_variance: Observation noise variance.
    Returns:
        mu: Predictive mean.
        sigma: Predictive standard deviation.
    """
    # Compute k(x, X)
    k_x_X = compute_k_x_X(x, X, length_scale, sigma_f)

    # Compute K
    K = compute_K(X, length_scale, sigma_f, noise_variance)

    # Compute K inverse
    K_inv = np.linalg.inv(K)

    # Predictive mean: mu(x) = k(x, X)^T * K^-1 * y
    mu = k_x_X.T @ K_inv @ y

    # Predictive variance: sigma^2(x) = k(x, x) - k(x, X)^T * K^-1 * k(x, X)
    k_x_x = squared_exponential_kernel(x, x, length_scale, sigma_f)
    var = k_x_x - k_x_X.T @ K_inv @ k_x_X

    # Ensure variance is non-negative (numerical errors)
    var = max(var, 0)
    sigma = np.sqrt(var)

    return mu, sigma

# Example usage
if __name__ == "__main__":
    # Observed data
    X = np.array([0.1, 0.5, 0.9])
    y = np.array([8.0, 10.0, 7.0])
    x_new = 0.6
    length_scale = 0.3
    sigma_f = 1.0
    noise_variance = 0.01  # Small noise for numerical stability

    # Compute prediction
    mu, sigma = gp_predict(x_new, X, y, length_scale, sigma_f, noise_variance)
    print(f"Predictive mean at x={x_new}: {mu:.3f}")
    print(f"Predictive standard deviation at x={x_new}: {sigma:.3f}")

Predictive mean at x=0.6: 9.602
Predictive standard deviation at x=0.6: 0.209
